# ВРЕМЯ И СВОДНЫЕ ТАБЛИЦЫ

## Задача

В папке subsid (`shared/homeworks/python_ds_miniprojects/5_subsid`) находятся файлы (`tm_sales_1`, `tm_sales_2`, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): `FILIAL_ID`, `SUBS_ID`, `PROD_ID`, `ACT_DTTM`.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

1. сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале `SUBS_ID`, то нужно его добавить
2. поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
3. продажа не засчитывается, если отключение (`END_DTTM`) произошло меньше чем через 5 минут после подключения (`START_DTTM`)
4. если в файле с продажами встречается строка без указанного `SUBS_ID`, она пропускается

Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format
Пример содержания итогового файла (колонка difference это разница между `START_DTTM` и `END_DTTM`):

ACT_DTTM;FILIAL_ID;difference  
20-03-2020 14:59;1;255 days 09:01:

In [38]:
import pandas as pd
import os

In [27]:
# сначала извелчем данные с логами
path_to_logs = '../shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv'
logs = pd.read_csv(path_to_logs, sep = ';')

In [28]:
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [29]:
logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
dtype: object

In [30]:
# отформатируем дату
logs.START_DTTM = pd.to_datetime(logs.START_DTTM, format=('%d-%m-%Y %H:%M'))
logs.END_DTTM = pd.to_datetime(logs.END_DTTM, format=('%d-%m-%Y %H:%M'))
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


In [31]:
# создадаим колонку difference, в которой будет хранится время между подлкючением и отключением пользователя 
logs['difference'] = logs.END_DTTM - logs.START_DTTM 
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00,0 days 00:01:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


In [34]:
# отфильтруем пользователей, которые отключились менее чем через 5 мин после подключения
logs = logs.loc[logs.difference>'5m']
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


In [51]:
# считаем данные из всех оставшихся файлов в один датафрейм

path_to_files = '../shared/homeworks/python_ds_miniprojects/5_subsid/subsid'

id_df = pd.DataFrame()

for root, dirs, path in os.walk(path_to_files):
    for i in path:
        if i != 'prod_activations_logs.csv':
            final_path = path_to_files + '/' + i
            current_df = pd.read_csv(final_path, sep=';')
            id_df = pd.concat([id_df, current_df], ignore_index=True)

id_df.ACT_DTTM = pd.to_datetime(id_df.ACT_DTTM, format=('%d-%m-%Y %H:%M'))
id_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
4,NaN,4,7421,2020-03-12 11:25:00
5,8362218,7,9879,2020-03-05 11:42:00
6,2185490,2,3210,2020-03-16 16:28:00
7,id5764122,3,1499,2020-03-18 15:44:00
8,id7642700,6,3020,2020-03-15 14:21:00
9,id1374509,2,5677,2020-03-17 11:48:00


In [52]:
# избавимся от строк, для которых неизвестен ID
id_df = id_df.dropna(subset=['SUBS_ID'])
id_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
5,8362218,7,9879,2020-03-05 11:42:00
6,2185490,2,3210,2020-03-16 16:28:00
7,id5764122,3,1499,2020-03-18 15:44:00
8,id7642700,6,3020,2020-03-15 14:21:00
9,id1374509,2,5677,2020-03-17 11:48:00


In [53]:
# Если 'id' нет в начале SUBS_ID, то нужно его добавить
for i in id_df.SUBS_ID:
    if i.startswith('id') == False:
        i2 = 'id' + i
        id_df = id_df.replace(i, i2)
id_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,id7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
5,id8362218,7,9879,2020-03-05 11:42:00
6,id2185490,2,3210,2020-03-16 16:28:00
7,id5764122,3,1499,2020-03-18 15:44:00
8,id7642700,6,3020,2020-03-15 14:21:00
9,id1374509,2,5677,2020-03-17 11:48:00


In [54]:
#соединим данные о логах с данными о пользователях
final_df = logs.merge(id_df)
final_df

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00,1,2020-03-20 14:59:00
1,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00,3,2020-03-17 10:17:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00,2,2020-03-16 16:28:00
3,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00,6,2020-03-15 14:21:00


In [60]:
list = [i for i in final_df.SUBS_ID.sort_values(ascending=True)]
list

['id2185490', 'id4651830', 'id5416547', 'id7642700']

In [61]:
# сохраним результат в запрошенном формате
final_df[['ACT_DTTM', 'FILIAL_ID', 'difference']].to_csv('Miniproject_5.2_result2')